# （1）Crate A and X matrix


In [2]:
import pandas as pd
import numpy as np

OD=pd.read_csv('D:\\GNN\\OCD\\OD_pd.csv')

In [3]:
OD.O_id.max()

2084

In [4]:
OD

,Unnamed: 0,O,D,flow,O_id,D_id
0,0,270539800001,270539800001,7255,1485,1485
1,1,270530269092,270530269092,3349,1159,1159
2,2,271390803011,271390803011,2460,1904,1904
3,3,270530251002,270530251002,2371,861,861
4,4,271630710181,271630710181,2102,2057,2057
...,...,...,...,...,...,...
262058,262058,270530259033,270530267104,1,917,1071
262059,262059,270530084001,270531052012,1,572,1309
262060,262060,270530003003,270531007001,1,524,1228
262061,262061,271230367003,270530249022,1,1682,857


In [40]:
m = n = 2085
Mat = [[0] * m] * n

In [41]:
Mat_np=np.array(Mat)

In [42]:
Mat_np

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [43]:
for i in range(len(OD)):
    O=OD.iloc[i,:]['O_id']
    D=OD.iloc[i,:]['D_id']
    Mat_np[O,D]=OD.iloc[i,:]['flow']
    if(Mat_np[O,D]>5000):
        print(Mat_np[O,D])


7255


In [44]:
Mat_np

array([[521,  85, 118, ...,   0,   0,   0],
       [100, 481, 159, ...,   0,   0,   0],
       [127, 170, 278, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ..., 172,  26,  11],
       [  0,   0,   0, ...,  67, 224,  25],
       [  0,   0,   0, ...,  20,  38, 105]])

In [45]:
# Diag is zero
row, col = np.diag_indices_from(Mat_np)
Mat_np[row,col] = 0

In [47]:
Mat_np

array([[  0,  85, 118, ...,   0,   0,   0],
       [100,   0, 159, ...,   0,   0,   0],
       [127, 170,   0, ...,   0,   0,   0],
       ...,
       [  0,   0,   0, ...,   0,  26,  11],
       [  0,   0,   0, ...,  67,   0,  25],
       [  0,   0,   0, ...,  20,  38,   0]])

In [ ]:
from scipy.sparse import csr_matrix

# convert to sparse matrix representation 
A = csr_matrix(Mat_np)
print("Sparse matrix: \n",A)

In [49]:
m = 1
n=2085
X = [[1] * m] * n

X_np=np.array(X)

X = csr_matrix(X_np)
print("Sparse matrix: \n",X)

Sparse matrix: 
   (0, 0)	1
  (1, 0)	1
  (2, 0)	1
  (3, 0)	1
  (4, 0)	1
  (5, 0)	1
  (6, 0)	1
  (7, 0)	1
  (8, 0)	1
  (9, 0)	1
  (10, 0)	1
  (11, 0)	1
  (12, 0)	1
  (13, 0)	1
  (14, 0)	1
  (15, 0)	1
  (16, 0)	1
  (17, 0)	1
  (18, 0)	1
  (19, 0)	1
  (20, 0)	1
  (21, 0)	1
  (22, 0)	1
  (23, 0)	1
  (24, 0)	1
  :	:
  (2060, 0)	1
  (2061, 0)	1
  (2062, 0)	1
  (2063, 0)	1
  (2064, 0)	1
  (2065, 0)	1
  (2066, 0)	1
  (2067, 0)	1
  (2068, 0)	1
  (2069, 0)	1
  (2070, 0)	1
  (2071, 0)	1
  (2072, 0)	1
  (2073, 0)	1
  (2074, 0)	1
  (2075, 0)	1
  (2076, 0)	1
  (2077, 0)	1
  (2078, 0)	1
  (2079, 0)	1
  (2080, 0)	1
  (2081, 0)	1
  (2082, 0)	1
  (2083, 0)	1
  (2084, 0)	1


In [50]:
from scipy import sparse



A.save_npz('A.npz', A)  #保存
X.save_npz('X.npz', X)  #保存

#csr_matrix_variable = sparse.load_npz('path.npz') #读



array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]], dtype=int32)

# Train the Overlapping GCN model

In [51]:
import os

os.chdir('D:/GNN/overlapping-community-detection-master/')#更改路径，''里面为更改的路径

import nocd
import matplotlib.pyplot as plt
import numpy as np
import scipy.sparse as sp
import torch
import torch.nn as nn
import torch.nn.functional as F

from sklearn.preprocessing import normalize


%matplotlib inline

torch.set_default_tensor_type(torch.cuda.FloatTensor)

Define the hyperparameters

In [54]:
hidden_sizes = [128]    # hidden sizes of the GNN
weight_decay = 1e-2     # strength of L2 regularization on GNN weights
dropout = 0.5           # whether to use dropout
batch_norm = True       # whether to use batch norm
lr = 1e-3               # learning rate
max_epochs = 500        # number of epochs to train
display_step = 25       # how often to compute validation loss
balance_loss = True     # whether to use balanced loss
stochastic_loss = True  # whether to use stochastic or full-batch training
batch_size = 20000      # batch size (only for stochastic training)

Select & normalize the feature matrix

For some datasets where the features are very informative / correlated with the community structure it's better to use `X` as input (e.g. co-authorship networks w/ keywords as node features). Otherwise, you should try using `A` or `[A, X]` as input.

In [55]:
x_norm = normalize(X)  # node features
# x_norm = normalize(A)  # adjacency matrix
# x_norm = sp.hstack([normalize(X), normalize(A)])  # concatenate A and X
x_norm = nocd.utils.to_sparse_tensor(x_norm).cuda()

Define the GNN model

In [56]:
N=X.shape[0]
K=50

sampler = nocd.sampler.get_edge_sampler(A, batch_size, batch_size, num_workers=5)
gnn = nocd.nn.GCN(x_norm.shape[1], hidden_sizes, K, batch_norm=batch_norm, dropout=dropout).cuda()
adj_norm = gnn.normalize_adj(A)
decoder = nocd.nn.BerpoDecoder(N, A.nnz, balance_loss=balance_loss)
opt = torch.optim.Adam(gnn.parameters(), lr=lr)

In [ ]:
val_loss = np.inf
validation_fn = lambda: val_loss
early_stopping = nocd.train.NoImprovementStopping(validation_fn, patience=10)
model_saver = nocd.train.ModelSaver(gnn)


for epoch, batch in enumerate(sampler):
    if epoch > max_epochs:
        break
    if epoch % 2 == 0:
        localtime = time.asctime( time.localtime(time.time()) )
        print(localtime)
        with torch.no_grad():
            gnn.eval()
            # Compute validation loss
            Z = F.relu(gnn(x_norm, adj_norm))
            val_loss = decoder.loss_full(Z, A)
            #print(f'Epoch {epoch:4d}, loss.full = {val_loss:.4f}, nmi = {get_nmi():.2f}')
            
            # Check if it's time for early stopping / to save the model
            early_stopping.next_step()
            if early_stopping.should_save():
                model_saver.save()
            if early_stopping.should_stop():
                print(f'Breaking due to early stopping at epoch {epoch}')
                break
            
    # Training step
    gnn.train()
    opt.zero_grad()
    Z = F.relu(gnn(x_norm, adj_norm))
    ones_idx, zeros_idx = batch
    if stochastic_loss:
        loss = decoder.loss_batch(Z, ones_idx, zeros_idx)
    else:
        loss = decoder.loss_full(Z, A)
    loss += nocd.utils.l2_reg_loss(gnn, scale=weight_decay)
    loss.backward()
    opt.step()

In [ ]:
print(AS)